In [11]:
import numpy as np
import pandas as pd

import os
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import operator 
import pandas as pd
import string
import re
import numpy as np
import sys
import os
import requests
import nltk
import pickle


init_notebook_mode(connected=True) #do not miss this line

from gensim import corpora, models, similarities

import warnings
warnings.filterwarnings("ignore")

In [12]:
main = pd.read_csv('train.csv', usecols=['label', 'tweet'])

In [13]:
main.describe()

,label
count,31962.000000
mean,0.070146
std,0.255397
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [14]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')
    
    return text

def strip_all_entities(text):
    """
    Remove twitter handles and hashtags
    """
    
    entity_prefixes = ['_', '@','#']
    for separator in string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)

    return ' '.join(words)

def strip_emojis(text):
    """
    Remove emojis and other symbols
    """
    
    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji.sub(r'', text)
    
    return text


def remove_stop_words(text):
    
    punctuation = set(string.punctuation)
    numbers = set(string.digits)
    stop_words = set(stopwords.words('english') + ['amp', 'via', 'https', 'Retweet', 'retweet', 'retweets', 'Retweets', 'RT']) 
    words = []
    for word in text.split():
        if word not in (stop_words or punctuation or numbers):
            words.append(word.lower())
            
    return ' '.join(words)

def remove_numbers(text):
    words = []
    for word in text.split():
        if word.isalpha() == True:
            words.append(word)
        else:
            continue
    
    return ' '.join(words)



main['tweet'] = main['tweet'].apply(strip_links)
main['tweet'] = main['tweet'].apply(strip_all_entities)
main['tweet'] = main['tweet'].apply(strip_emojis)
main['tweet'] = main['tweet'].apply(remove_stop_words)
main['tweet'] = main['tweet'].apply(remove_numbers)

In [19]:
def reclassifier(c):
    if c == 0:
        instance = 2
    elif c == 1:
        instance = 1
    
    return instance

main['label'] = main['label'].apply(reclassifier)

In [27]:
main = main.rename(columns={'label':'class'})

In [28]:
main.head()

,class,tweet
0,2,father dysfunctional selfish drags kids dysfun...
1,2,thanks credit use cause offer wheelchair vans pdx
2,2,bihday majesty
3,2,love u take u time
4,2,factsguide society


In [30]:
main[main['class']==1].to_csv('cleanNegData.csv')